# Building a Multi-Agent System using Gemini API

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand the fundamentals of multi-agent systems
- ✅ Master API key integration with Google Gemini API (both Colab and local environments)
- ✅ Learn how to orchestrate multiple AI agents to work together
- ✅ Build a complete AI Research Assistant with three specialized agents
- ✅ Understand prompt engineering for agent specialization
- ✅ Learn error handling and best practices for production systems
- ✅ Explore test cases and additional scenarios
- ✅ Design a full-stack solution architecture

## 📚 What You'll Build

A sophisticated **AI Research Assistant** composed of three specialized agents:
1. **The Planner Agent**: Breaks down complex topics into researchable questions
2. **The Researcher Agent**: Searches the web for information using Google Search
3. **The Synthesizer Agent**: Compiles research into a comprehensive report

## 🧠 Key Concepts You'll Learn

1. **Multi-Agent Systems**: How multiple AI agents collaborate to solve complex problems
2. **Agent Orchestration**: Coordinating agents in a workflow
3. **API Key Management**: Secure handling of API credentials
4. **Prompt Engineering**: Designing effective prompts for specific agent roles
5. **Tool Integration**: Using external tools (Google Search) with LLMs
6. **Error Handling**: Building robust systems that handle failures gracefully

---

## ⚠️ Prerequisites

Before starting, make sure you have:
1. A Google account (for Gemini API access)
2. A Gemini API key ([Get it here](https://makersuite.google.com/app/apikey))
3. Python 3.8+ installed
4. Required packages installed (see next cell)

**Note**: This notebook works in both Google Colab and local Jupyter environments. We'll show you how to configure API keys for both scenarios.


## 📦 Step 1: Install Required Packages

First, let's install the Google GenAI library. This is the official Python SDK for the Gemini API.

**⚠️ Important Note**: This notebook uses the new `google-genai` package, which replaces the deprecated `google-generativeai` package. The new package provides better support and access to the latest features.


In [1]:
# Install the Google GenAI library (new package - replaces deprecated google-generativeai)
# %pip install google-genai -q

# Verify installation
try:
    import google.genai as genai
    print("✅ google-genai installed successfully!")
    print(f"📦 Version: {genai.version if hasattr(genai, 'version') else 'N/A'}")
    print("📝 Note: Using the new google-genai package (google-generativeai is deprecated)")
except ImportError as e:
    print(f"❌ Error importing google-genai: {e}")
    print("Please run: pip install google-genai")


✅ google-genai installed successfully!
📦 Version: <module 'google.genai.version' from '/Users/balaji/Documents/Learning/AI/ai_agent_projects/venv/lib/python3.13/site-packages/google/genai/version.py'>
📝 Note: Using the new google-genai package (google-generativeai is deprecated)


## 🔑 Step 2: API Key Configuration

**This is one of the most important steps!** Proper API key management is crucial for:
- Security: Never expose your API keys in code
- Flexibility: Support different environments (Colab, local, production)
- Best Practices: Follow industry standards for credential management

### Understanding API Key Security

⚠️ **NEVER commit API keys to version control (Git)**
- API keys are like passwords - they give access to your account
- Exposed keys can be used by others, leading to unexpected charges
- Always use environment variables or secure storage

### Two Configuration Methods

We'll show you both methods:
1. **Google Colab**: Using Colab's built-in Secrets feature
2. **Local Environment**: Using environment variables or `.env` files


In [2]:

import os
import google.genai as genai
from typing import Optional

# ============================================================================
# GLOBAL CONFIGURATION: Change the model here to test different models
# ============================================================================
# Default model for all agents. Change this to test different models:
# - 'gemini-2.5-flash' - Recommended for free tier (fast, efficient)
# - 'gemini-2.0-flash-exp' - Experimental Flash model
# - 'gemini-1.5-pro' - Requires paid tier
# - 'gemini-2.5-pro' - Requires paid tier
DEFAULT_MODEL = 'gemini-2.5-flash'

def configure_gemini(api_key: Optional[str] = None) -> genai.Client:
    """
    Configure the Gemini API with secure API key management.
    
    This function supports multiple methods for API key retrieval:
    1. Direct parameter (for testing)
    2. Environment variable (recommended for local development)
    3. Google Colab Secrets (for Colab notebooks)
    
    Args:
        api_key: Optional API key string. If not provided, will try to get from:
                 - GOOGLE_API_KEY environment variable
                 - Colab userdata secrets (if in Colab)
    
    Note: The model is selected via DEFAULT_MODEL constant at the top of this cell.
          All agent functions will use DEFAULT_MODEL unless you explicitly pass a model_name.
    
    Returns:
        Configured genai.Client instance
    
    Raises:
        ValueError: If API key cannot be found
    """
    # Method 1: Use provided API key (for testing)
    if api_key:
        client = genai.Client(api_key=api_key)
        print("✅ API key configured from parameter")
        return client
    
    # Method 2: Try environment variable (recommended for local)
    api_key = os.getenv('GOOGLE_API_KEY')
    if api_key:
        client = genai.Client(api_key=api_key)
        print("✅ API key configured from environment variable")
        return client
    
    # Method 3: Try Colab Secrets (only works in Colab)
    try:
        from google.colab import userdata
        secret_name = "GOOGLE_API_KEY"
        api_key = userdata.get(secret_name)
        if api_key:
            client = genai.Client(api_key=api_key)
            print("✅ API key configured from Colab Secrets")
            return client
    except ImportError:
        # Not in Colab, continue to error
        pass
    
    # If we get here, no API key was found
    raise ValueError(
        "API key not found! Please provide it using one of these methods:\n"
        "1. Set environment variable: export GOOGLE_API_KEY='your-key-here'\n"
        "2. In Colab: Add 'GOOGLE_API_KEY' to Secrets (🔑 icon in sidebar)\n"
        "3. Pass directly: configure_gemini(api_key='your-key-here')\n\n"
        "Get your API key from: https://makersuite.google.com/app/apikey"
    )

# Test the configuration (will fail if no key is set - that's expected!)
print("📝 Configuration function loaded. Ready to configure API key.")
print(f"📦 Default model for all agents: {DEFAULT_MODEL}")
print("\n💡 To change the model for ALL agents in the notebook:")
print(f"   1. Edit DEFAULT_MODEL at the top of this cell (currently: '{DEFAULT_MODEL}')")
print("   2. Re-run this cell to reload the configuration")
print("   3. All agent functions will automatically use the new model")
print("\n💡 To use this function:")
print("   - Local: Set GOOGLE_API_KEY environment variable")
print("   - Colab: Add GOOGLE_API_KEY to Secrets")
print("   - Testing: Pass api_key parameter directly")


📝 Configuration function loaded. Ready to configure API key.
📦 Default model for all agents: gemini-2.5-flash

💡 To change the model for ALL agents in the notebook:
   1. Edit DEFAULT_MODEL at the top of this cell (currently: 'gemini-2.5-flash')
   2. Re-run this cell to reload the configuration
   3. All agent functions will automatically use the new model

💡 To use this function:
   - Local: Set GOOGLE_API_KEY environment variable
   - Colab: Add GOOGLE_API_KEY to Secrets
   - Testing: Pass api_key parameter directly


### ⚠️ Important: Free Tier Model Restrictions

**Free Tier Limitations (as of 2025):**

- ✅ **Available on Free Tier:**
  - `gemini-2.5-flash` - Fast, efficient model (recommended for free tier) ⭐
  - `gemini-2.0-flash-exp` - Experimental Flash model
  - Limited quotas (typically 15 requests per minute)

- ❌ **NOT Available on Free Tier:**
  - `gemini-1.5-pro` - Requires paid tier
  - `gemini-1.5-flash` - No longer available on free tier
  - `gemini-2.5-pro` - Requires paid tier
  - Most "pro" models require a paid subscription

**💡 Recommendation:** Use `gemini-2.5-flash` as your default model for free tier accounts. It's fast, capable, and free!

**To check your available models, run the model listing cell below.**


### 🔧 API Usage Helper Function

The new `google.genai` API uses `client.models.generate_content()` instead of `chats.create().send_message()`. 
Below is a helper function that works correctly with both free and paid tiers:


In [3]:
def generate_content_safe(client: genai.Client, model_name: str = None, prompt: str = ""):
    """
    Helper function to generate content using the correct API method.
    Works with both free and paid tier models.
    
    Args:
        client: Configured genai.Client instance
        model_name: Model name (defaults to DEFAULT_MODEL if not provided)
        prompt: The prompt text
    
    Returns:
        Response text as string, or None on error
    """
    try:
        # Use DEFAULT_MODEL if model_name not provided
        if model_name is None:
            model_name = DEFAULT_MODEL
        
        # Use models.generate_content() - the correct method
        response = client.models.generate_content(
            model=model_name,
            contents=prompt
        )
        
        # Extract text from response - handle different response formats
        if hasattr(response, 'text'):
            return response.text
        elif hasattr(response, 'candidates') and len(response.candidates) > 0:
            # Standard response format
            candidate = response.candidates[0]
            if hasattr(candidate, 'content') and hasattr(candidate.content, 'parts'):
                if len(candidate.content.parts) > 0:
                    return candidate.content.parts[0].text
        elif hasattr(response, 'content') and hasattr(response.content, 'text'):
            return response.content.text
        
        # Fallback: try to convert to string
        return str(response)
        
    except Exception as e:
        error_msg = str(e)
        if "404" in error_msg or "not found" in error_msg.lower():
            print(f"❌ Model '{model_name}' not found. This model may not be available on free tier.")
            print(f"💡 Try using '{DEFAULT_MODEL}' instead, or change DEFAULT_MODEL in the configuration cell.")
        elif "quota" in error_msg.lower() or "limit" in error_msg.lower():
            print(f"❌ Quota exceeded. Please check your API usage limits.")
        else:
            print(f"❌ Error: {error_msg}")
        return None

# Test the helper function
print("✅ Helper function loaded. Use generate_content_safe() for reliable API calls.")


✅ Helper function loaded. Use generate_content_safe() for reliable API calls.


### 🔐 Setting Up Your API Key (Choose Your Environment)

#### Option A: Google Colab Setup

1. Click the **🔑 (Key)** icon in the left sidebar
2. Click **"Add new secret"**
3. Name: `GOOGLE_API_KEY`
4. Value: Your API key from [Google AI Studio](https://makersuite.google.com/app/apikey)
5. Click **"Add secret"**

The code above will automatically detect and use it!

#### Option B: Local Jupyter Notebook Setup

**Method 1: Environment Variable (Recommended)**
```bash
# In your terminal (before starting Jupyter):
export GOOGLE_API_KEY='your-api-key-here'

# Or add to your ~/.bashrc or ~/.zshrc for persistence:
echo 'export GOOGLE_API_KEY="your-api-key-here"' >> ~/.zshrc
source ~/.zshrc
```

**Method 2: Using .env file (Alternative)**
```python
# Install python-dotenv first: pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()  # Loads .env file
```

**Method 3: Direct Input (For Testing Only)**
```python
# ⚠️ Only for testing! Never commit this to Git!
client = configure_gemini(api_key='your-api-key-here')
```

### 🧪 Test Your Configuration

Run the cell below to test if your API key is configured correctly:


In [4]:
from dotenv import load_dotenv
load_dotenv()

# Test API key configuration    
client = configure_gemini()
# Use DEFAULT_MODEL from configuration cell
response = client.models.generate_content(model=DEFAULT_MODEL, contents="Say 'Hello' in one word.")
# Extract response text
if hasattr(response, 'text'):
    print(response.text)
elif hasattr(response, 'candidates') and len(response.candidates) > 0:
    print(response.candidates[0].content.parts[0].text)
else:
    print(str(response))


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


✅ API key configured from environment variable
Hello


In [5]:
print(response)

sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text='Hello'
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>,
  index=0
)] create_time=None model_version='gemini-2.5-flash' prompt_feedback=None response_id='GI1cacSDMITPz7IPlf-xqAw' usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=1,
  prompt_token_count=9,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=9
    ),
  ],
  thoughts_token_count=45,
  total_token_count=55
) automatic_function_calling_history=[] parsed=None


### 📋 List Available Gemini Models

Use the cell below to see all available Gemini models and their capabilities:


In [6]:
def list_gemini_models(client: genai.Client = None, show_details: bool = True):
    """
    List all available Gemini models.
    
    Args:
        client: Optional genai.Client instance. If not provided, will try to configure one.
        show_details: If True, shows detailed information about each model
    
    Returns:
        List of available model names
    """
    try:
        # Get client if not provided
        if client is None:
            client = configure_gemini()
        
        print("🔍 Fetching available Gemini models...\n")
        
        # List all models
        models = client.models.list()
        
        # Filter for Gemini models (they typically start with 'gemini')
        gemini_models = [m for m in models if 'gemini' in m.name.lower()]
        
        if not gemini_models:
            print("⚠️  No Gemini models found. Showing all available models:")
            gemini_models = list(models)
        
        print(f"✅ Found {len(gemini_models)} available model(s):\n")
        print("=" * 80)
        
        model_names = []
        for i, model in enumerate(gemini_models, 1):
            model_name = model.name.split('/')[-1] if '/' in model.name else model.name
            model_names.append(model_name)
            
            print(f"\n{i}. Model: {model_name}")
            
            if show_details:
                # Display model details if available
                if hasattr(model, 'display_name') and model.display_name:
                    print(f"   Display Name: {model.display_name}")
                
                if hasattr(model, 'description') and model.description:
                    print(f"   Description: {model.description}")
                
                if hasattr(model, 'supported_generation_methods'):
                    methods = model.supported_generation_methods
                    if methods:
                        print(f"   Supported Methods: {', '.join(methods)}")
                
                if hasattr(model, 'input_token_limit'):
                    print(f"   Input Token Limit: {model.input_token_limit:,}")
                
                if hasattr(model, 'output_token_limit'):
                    print(f"   Output Token Limit: {model.output_token_limit:,}")
        
        print("\n" + "=" * 80)
        print(f"\n💡 To use a model, pass its name to the agent functions:")
        print(f"   Example: planner_agent(client, topic, model_name='{model_names[0] if model_names else 'gemini-2.5-flash'}')")
        
        return model_names
        
    except Exception as e:
        print(f"❌ Error listing models: {e}")
        import traceback
        traceback.print_exc()
        return []

# List available models
# Make sure you've configured your API key first!
try:
    client = configure_gemini()
    available_models = list_gemini_models(client)
except Exception as e:
    print(f"❌ Error: {e}")
    print("Make sure you've configured your API key in the previous cell!")


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


✅ API key configured from environment variable
🔍 Fetching available Gemini models...

✅ Found 32 available model(s):


1. Model: gemini-2.5-flash
   Display Name: Gemini 2.5 Flash
   Description: Stable version of Gemini 2.5 Flash, our mid-size multimodal model that supports up to 1 million tokens, released in June of 2025.
   Input Token Limit: 1,048,576
   Output Token Limit: 65,536

2. Model: gemini-2.5-pro
   Display Name: Gemini 2.5 Pro
   Description: Stable release (June 17th, 2025) of Gemini 2.5 Pro
   Input Token Limit: 1,048,576
   Output Token Limit: 65,536

3. Model: gemini-2.0-flash-exp
   Display Name: Gemini 2.0 Flash Experimental
   Description: Gemini 2.0 Flash Experimental
   Input Token Limit: 1,048,576
   Output Token Limit: 8,192

4. Model: gemini-2.0-flash
   Display Name: Gemini 2.0 Flash
   Description: Gemini 2.0 Flash
   Input Token Limit: 1,048,576
   Output Token Limit: 8,192

5. Model: gemini-2.0-flash-001
   Display Name: Gemini 2.0 Flash 001
   Descriptio

### 🚀 Quick Reference: Simple Model List

For a quick list of just model names:


In [7]:
# Quick list of available Gemini models
try:
    client = configure_gemini()
    models = client.models.list()
    
    # Extract just the model names (remove path prefix)
    gemini_models = [
        m.name.split('/')[-1] 
        for m in models 
        if 'gemini' in m.name.lower()
    ]
    
    if gemini_models:
        print("📋 Available Gemini Models:")
        print("-" * 40)
        for model in gemini_models:
            print(f"  • {model}")
        print("-" * 40)
        print(f"\n💡 Total: {len(gemini_models)} model(s) available")
    else:
        print("⚠️  No Gemini models found. Check your API key and permissions.")
        
except Exception as e:
    print(f"❌ Error: {e}")
    print("Make sure you've configured your API key!")


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


✅ API key configured from environment variable
📋 Available Gemini Models:
----------------------------------------
  • gemini-2.5-flash
  • gemini-2.5-pro
  • gemini-2.0-flash-exp
  • gemini-2.0-flash
  • gemini-2.0-flash-001
  • gemini-2.0-flash-exp-image-generation
  • gemini-2.0-flash-lite-001
  • gemini-2.0-flash-lite
  • gemini-2.0-flash-lite-preview-02-05
  • gemini-2.0-flash-lite-preview
  • gemini-exp-1206
  • gemini-2.5-flash-preview-tts
  • gemini-2.5-pro-preview-tts
  • gemini-flash-latest
  • gemini-flash-lite-latest
  • gemini-pro-latest
  • gemini-2.5-flash-lite
  • gemini-2.5-flash-image-preview
  • gemini-2.5-flash-image
  • gemini-2.5-flash-preview-09-2025
  • gemini-2.5-flash-lite-preview-09-2025
  • gemini-3-pro-preview
  • gemini-3-flash-preview
  • gemini-3-pro-image-preview
  • gemini-robotics-er-1.5-preview
  • gemini-2.5-computer-use-preview-10-2025
  • gemini-embedding-exp-03-07
  • gemini-embedding-exp
  • gemini-embedding-001
  • gemini-2.5-flash-native-audio

In [8]:
# Test API key configuration
try:
    client = configure_gemini()
    print("🎉 Configuration successful! Your API key is working.")
    
    # Quick test: Generate a simple response using the correct API method
    # Use models.generate_content() - the correct method for new API
    # Uses DEFAULT_MODEL from the configuration cell
    response = client.models.generate_content(
        model=DEFAULT_MODEL,
        contents="Say 'Hello' in one word."
    )
    
    # Extract text from response
    if hasattr(response, 'text'):
        response_text = response.text
    elif hasattr(response, 'candidates') and len(response.candidates) > 0:
        response_text = response.candidates[0].content.parts[0].text
    else:
        response_text = str(response)
    
    print(f"✅ API Test Response: {response_text}")
    
except ValueError as e:
    print(f"❌ Configuration Error:\n{e}")
    print("\n📖 Please follow the setup instructions above to configure your API key.")
except Exception as e:
    print(f"❌ Unexpected Error: {e}")
    print("This might indicate an invalid API key or network issue.")
    import traceback
    traceback.print_exc()


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


✅ API key configured from environment variable
🎉 Configuration successful! Your API key is working.
✅ API Test Response: Hello


## 🤖 Understanding Multi-Agent Systems

### What is a Multi-Agent System?

A **Multi-Agent System (MAS)** is a system composed of multiple autonomous agents that:
- Work together to solve complex problems
- Each agent has a specific role/expertise
- Agents communicate and pass information between each other
- The system is more capable than any single agent alone

### Why Use Multi-Agent Systems?

1. **Specialization**: Each agent can be optimized for a specific task
2. **Modularity**: Easy to modify or replace individual agents
3. **Scalability**: Can add more agents as needed
4. **Reliability**: If one agent fails, others can continue
5. **Complexity Management**: Break complex problems into manageable pieces

### Our Architecture

```
┌─────────────┐
│   User      │
│  (Topic)    │
└──────┬──────┘
       │
       ▼
┌─────────────────┐
│  Planner Agent  │  ← Breaks topic into questions
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│ Researcher Agent│  ← Searches for each question
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│Synthesizer Agent│  ← Combines into final report
└────────┬────────┘
         │
         ▼
┌─────────────┐
│ Final Report│
└─────────────┘
```

### Key Design Principles

1. **Single Responsibility**: Each agent does ONE thing well
2. **Clear Interfaces**: Agents communicate through well-defined data structures
3. **Error Handling**: Each agent handles its own errors gracefully
4. **Orchestration**: A main function coordinates the workflow


## 🎯 Agent 1: The Planner Agent

### Role and Responsibilities

The **Planner Agent** is like a project manager. Its job is to:
- Take a broad, complex topic
- Break it down into 3-5 specific, researchable questions
- Ensure questions are answerable and focused
- Provide structure to the research process

### Why We Need a Planner

A vague topic like "climate change" is too broad. The Planner breaks it into:
- "What are the main causes of climate change?"
- "How does climate change affect sea levels?"
- "What are the economic impacts of climate change?"
- etc.

### Prompt Engineering Strategy

Notice how we:
1. **Define the persona**: "You are an expert research planner"
2. **Specify the task**: "Break down the topic into questions"
3. **Constrain the output**: "Return as Python list of strings"
4. **Provide examples**: Show the expected format

This is **prompt engineering** - a critical skill in AI development!


In [9]:
def planner_agent(client: genai.Client, topic: str, model_name: str = None) -> list[str]:
    """
    Planner Agent: Breaks down a broad topic into specific research questions.
    
    This agent uses prompt engineering to ensure the LLM:
    - Understands its role (expert research planner)
    - Follows the task (break down topic)
    - Returns structured output (list of strings)
    
    Args:
        client: Configured genai.Client instance
        topic: The broad research topic to break down
        model_name: Optional model name. If not provided, uses DEFAULT_MODEL global constant.
                    Change DEFAULT_MODEL in the configuration cell to test different models.
    
    Returns:
        List of specific research questions (strings)
        Returns empty list on error
    
    Example:
        >>> client = configure_gemini()
        >>> questions = planner_agent(client, "Artificial Intelligence")
        >>> print(questions)
        ['What is the history of AI?', 'What are current AI applications?', ...]
    """
    # Use DEFAULT_MODEL if model_name not provided
    if model_name is None:
        model_name = DEFAULT_MODEL
    
    print("📋 Planner Agent: Creating a research plan...")
    print(f"   Using model: {model_name}")
    
    # Carefully crafted prompt with clear instructions
    prompt = f"""
    You are an expert research planner. Your task is to break down the following topic
    into 3-5 specific, answerable questions. These questions should be:
    - Focused and researchable
    - Cover different aspects of the topic
    - Suitable for web search
    
    TOPIC: "{topic}"
    
    Return ONLY a Python list of strings in this exact format:
    ["question 1", "question 2", "question 3"]
    
    Do not include any other text, explanations, or markdown formatting.
    """
    
    try:
        # Use models.generate_content() - the correct method for new API
        response = client.models.generate_content(
            model=model_name,
            contents=prompt
        )
        
        # Extract text from response
        if hasattr(response, 'text'):
            plan_text = response.text.strip()
        elif hasattr(response, 'candidates') and len(response.candidates) > 0:
            # Standard response format
            plan_text = response.candidates[0].content.parts[0].text.strip()
        elif hasattr(response, 'content') and hasattr(response.content, 'text'):
            plan_text = response.content.text.strip()
        else:
            # Fallback: try to convert to string
            plan_text = str(response).strip()
        
        # Remove markdown code blocks if present
        if plan_text.startswith("```"):
            plan_text = plan_text.split("```")[1]
            if plan_text.startswith("python") or plan_text.startswith("json"):
                plan_text = plan_text.split("\n", 1)[1]
        
        # Extract the list
        # Remove brackets and quotes, then split by comma
        plan_text = plan_text.replace('[', '').replace(']', '').strip()
        # Handle both single and double quotes
        plan_text = plan_text.replace('"', '').replace("'", '')
        
        # Split by comma and clean up
        plan = [q.strip() for q in plan_text.split(',') if q.strip()]
        
        # Validate we got questions
        if not plan:
            print("⚠️  Warning: Planner returned empty plan. Trying alternative parsing...")
            # Fallback: try to extract questions from natural language
            lines = plan_text.split('\n')
            plan = [line.strip().lstrip('- ').lstrip('* ').lstrip('1. ').lstrip('2. ').lstrip('3. ').lstrip('4. ').lstrip('5. ')
                   for line in lines if line.strip() and ('?' in line or line.strip().startswith('What') or line.strip().startswith('How'))]
            plan = [q for q in plan if len(q) > 10][:5]  # Filter and limit
        
        if plan:
            print(f"✅ Plan created with {len(plan)} questions:")
            for i, question in enumerate(plan, 1):
                print(f"   {i}. {question}")
        else:
            print("❌ Failed to parse research plan")
        
        return plan
        
    except Exception as e:
        print(f"❌ Error in Planner Agent: {e}")
        import traceback
        traceback.print_exc()
        return []


In [10]:
# Test the Planner Agent
# Make sure you've configured your API key in the earlier cell!

try:
    client = configure_gemini()
    test_topic = "Quantum Computing"
    print(f"🧪 Testing Planner Agent with topic: '{test_topic}'\n")
    
    questions = planner_agent(client, test_topic)
    
    if questions:
        print(f"\n✅ Success! Generated {len(questions)} research questions.")
    else:
        print("\n❌ Failed to generate questions. Check your API key and try again.")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Make sure you've configured your API key in Step 2!")


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


✅ API key configured from environment variable
🧪 Testing Planner Agent with topic: 'Quantum Computing'

📋 Planner Agent: Creating a research plan...
   Using model: gemini-2.5-flash
✅ Plan created with 13 questions:
   1. What are the fundamental principles and key components (e.g.
   2. qubits
   3. superposition
   4. entanglement) that enable quantum computing?
   5. What is the current state of quantum computing development
   6. and what are the primary technological and engineering challenges facing its widespread practical application?
   7. What are the most promising potential real-world applications of quantum computing across different industries
   8. and how might they impact existing technologies?
   9. What are the leading quantum computing hardware architectures (e.g.
   10. superconducting
   11. trapped ion
   12. photonic)
   13. and what are their respective advantages and disadvantages?

✅ Success! Generated 13 research questions.


## 🔍 Agent 2: The Researcher Agent

### Role and Responsibilities

The **Researcher Agent** is the information gatherer. Its job is to:
- Take a specific research question
- Use Google Search to find current, relevant information
- Return detailed answers based on web search results
- Provide factual, up-to-date information

### Key Concept: Tool Integration

This agent demonstrates a crucial concept: **Tool Integration**. The Gemini API supports "tools" - external capabilities that the model can use. In this case, we're giving the model access to Google Search.

### How Google Search Tool Works

1. We define a `Tool` object with `GoogleSearch` (note: `GoogleSearchRetrieval` is deprecated)
2. Pass it to `generate_content()` with the `config` parameter
3. The model automatically decides when to use the search tool
4. It retrieves real-time information from the web
5. Uses that information to generate a comprehensive answer

This is called **Retrieval-Augmented Generation (RAG)** - combining retrieval (search) with generation (LLM).


In [13]:
def search_agent(client: genai.Client, question: str, model_name: str = None) -> str:
    """
    Researcher Agent: Searches the web for information about a specific question.
    
    This agent uses Gemini's Google Search Retrieval tool to:
    - Access real-time web information
    - Find current, relevant sources
    - Generate comprehensive answers based on search results
    
    Args:
        client: Configured genai.Client instance
        question: The specific research question to answer
        model_name: Optional model name. If not provided, uses DEFAULT_MODEL global constant.
                    Change DEFAULT_MODEL in the configuration cell to test different models.
    
    Returns:
        Detailed answer string based on web search
        Returns empty string on error
    
    Example:
        >>> client = configure_gemini()
        >>> answer = search_agent(client, "What is quantum computing?")
        >>> print(answer)
        "Quantum computing is a type of computation that uses quantum..."
    """
    # Use DEFAULT_MODEL if model_name not provided
    if model_name is None:
        model_name = DEFAULT_MODEL
    
    print(f"🔍 Researcher Agent: Researching question: '{question}'...")
    print(f"   Using model: {model_name}")
    
    try:
        # Craft a prompt that encourages comprehensive research
        prompt = f"""
        Provide a detailed, well-researched answer to the following question.
        Use information from recent and authoritative sources.
        Include key facts, statistics, and explanations.
        
        Question: {question}
        
        Answer:
        """
        
        # Use models.generate_content() with Google Search tool enabled
        # The new API enables Google Search through the config parameter
        from google.genai import types
        
        config = types.GenerateContentConfig(
            tools=[types.Tool(
                google_search=types.GoogleSearch()
            )]
        )
        
        # Use models.generate_content() - the correct method for new API
        response = client.models.generate_content(
            model=model_name,
            contents=prompt,
            config=config
        )
        
        # Extract text from response
        if hasattr(response, 'text'):
            answer_text = response.text
        elif hasattr(response, 'candidates') and len(response.candidates) > 0:
            # Standard response format
            answer_text = response.candidates[0].content.parts[0].text
        elif hasattr(response, 'content') and hasattr(response.content, 'text'):
            answer_text = response.content.text
        else:
            answer_text = str(response)
        
        if answer_text:
            print(f"   ✅ Information found ({len(answer_text)} characters)")
            return answer_text
        else:
            print("   ⚠️  No text in response")
            return ""
            
    except Exception as e:
        print(f"   ❌ Error in Researcher Agent: {e}")
        import traceback
        traceback.print_exc()
        return ""


### 🧪 Test the Researcher Agent

Let's test the Researcher Agent with a sample question:


In [14]:
# Test the Researcher Agent
try:
    client = configure_gemini()
    test_question = "What are the latest developments in quantum computing in 2024?"
    print(f"🧪 Testing Researcher Agent with question: '{test_question}'\n")
    
    answer = search_agent(client, test_question)
    
    if answer:
        print(f"\n✅ Success! Retrieved answer ({len(answer)} characters)")
        print(f"\n📄 Answer Preview (first 500 chars):")
        print("-" * 60)
        print(answer[:500] + "..." if len(answer) > 500 else answer)
        print("-" * 60)
    else:
        print("\n❌ Failed to retrieve information. Check your API key and try again.")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Make sure you've configured your API key in Step 2!")


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


✅ API key configured from environment variable
🧪 Testing Researcher Agent with question: 'What are the latest developments in quantum computing in 2024?'

🔍 Researcher Agent: Researching question: 'What are the latest developments in quantum computing in 2024?'...
   Using model: gemini-2.5-flash
   ✅ Information found (6283 characters)

✅ Success! Retrieved answer (6283 characters)

📄 Answer Preview (first 500 chars):
------------------------------------------------------------
Quantum computing witnessed significant advancements in 2024, moving closer to practical applications across various sectors. Breakthroughs in hardware, software, algorithms, and increased investment are accelerating its development, although challenges like error rates and scalability persist.

**Key Developments and Breakthroughs in 2024:**

*   **Increased Qubit Stability and Error Correction:** A critical challenge in quantum computing has been maintaining the stability of qubits. In 2024, r...
------------

## ✍️ Agent 3: The Synthesizer Agent

### Role and Responsibilities

The **Synthesizer Agent** is the writer and compiler. Its job is to:
- Take all the research results from the Researcher Agent
- Combine them into a coherent, well-structured report
- Ensure the report covers all aspects of the original topic
- Create a professional, readable final document

### Key Design Principles

1. **Context Aggregation**: Combine all research notes into a single context
2. **Structured Output**: Generate a report with introduction, body, and conclusion
3. **Source Constraint**: Only use information from provided research (prevents hallucination)
4. **Quality Control**: Ensure coherence and flow

### Why "Use Research Notes ONLY"?

This constraint is **critical** for:
- **Accuracy**: Prevents the model from making up information
- **Traceability**: All information comes from the Researcher Agent
- **Reliability**: More predictable and verifiable outputs
- **Control**: You know exactly what sources were used


In [16]:
def synthesizer_agent(
    client: genai.Client, 
    topic: str, 
    research_results: list[tuple[str, str]],
    model_name: str = None
) -> str:
    """
    Synthesizer Agent: Combines research results into a comprehensive report.
    
    This agent takes all the fragmented research and weaves it into a coherent,
    well-structured report. It's critical that this agent ONLY uses the provided
    research notes to prevent hallucination.
    
    Args:
        client: Configured genai.Client instance
        topic: The original research topic
        research_results: List of tuples (question, research_data)
        model_name: Optional model name. If not provided, uses DEFAULT_MODEL global constant.
                    Change DEFAULT_MODEL in the configuration cell to test different models.
    
    Returns:
        Comprehensive research report as a string
        Returns error message on failure
    
    Example:
        >>> client = configure_gemini()
        >>> results = [("Q1", "Answer 1"), ("Q2", "Answer 2")]
        >>> report = synthesizer_agent(client, "AI", results)
        >>> print(report)
        "# Research Report: AI\n\n## Introduction\n..."
    """
    # Use DEFAULT_MODEL if model_name not provided
    if model_name is None:
        model_name = DEFAULT_MODEL
    
    print("✍️  Synthesizer Agent: Writing the final report...")
    print(f"   Using model: {model_name}")
    
    # Aggregate all research into a structured format
    # This becomes the context for the final synthesis
    research_notes = ""
    for i, (question, data) in enumerate(research_results, 1):
        research_notes += f"""
### Research Question {i}: {question}

Research Findings:
{data}

---
"""
    
    # Carefully crafted prompt with strict constraints
    prompt = f"""
    You are an expert research analyst and technical writer. Your task is to synthesize
    the provided research notes into a comprehensive, well-structured report on the topic: "{topic}".
    
    IMPORTANT CONSTRAINTS:
    1. Use ONLY the information provided in the research notes below
    2. Do NOT add information that is not in the research notes
    3. If information is missing, acknowledge it rather than making it up
    4. Maintain accuracy and cite the research questions when relevant
    
    REPORT STRUCTURE:
    - **Introduction**: Overview of the topic and what will be covered
    - **Main Body**: Organized sections covering key findings from the research
    - **Conclusion**: Summary of main points and key takeaways
    
    Write in a professional, clear, and engaging style suitable for a research report.
    
    ## Research Notes ##
    {research_notes}
    
    ## Report ##
    """
    
    try:
        # Use models.generate_content() - the correct method for new API
        response = client.models.generate_content(
            model=model_name,
            contents=prompt
        )
        
        # Extract text from response
        if hasattr(response, 'text'):
            report_text = response.text
        elif hasattr(response, 'candidates') and len(response.candidates) > 0:
            # Standard response format
            report_text = response.candidates[0].content.parts[0].text
        elif hasattr(response, 'content') and hasattr(response.content, 'text'):
            report_text = response.content.text
        else:
            report_text = str(response)
        
        if report_text:
            print(f"   ✅ Report generated ({len(report_text)} characters)")
            return report_text
        else:
            print("   ⚠️  No text in response")
            return "Error: Could not generate the final report."
            
    except Exception as e:
        print(f"   ❌ Error in Synthesizer Agent: {e}")
        import traceback
        traceback.print_exc()
        return "Error: Could not generate the final report."


## 🎼 The Orchestrator: Main Function

### What is Orchestration?

**Orchestration** is the coordination of multiple agents to work together in a workflow. The orchestrator:
- Manages the flow of data between agents
- Handles errors and edge cases
- Ensures each agent runs at the right time
- Collects and passes information correctly

### Our Orchestration Flow

```
1. Configure API → Get model
2. Get user input → Topic
3. Planner Agent → List of questions
4. For each question:
   └─> Researcher Agent → Research data
5. Collect all research → List of (question, data) tuples
6. Synthesizer Agent → Final report
7. Display report → User
```

### Error Handling Strategy

At each step, we check for errors:
- If configuration fails → Exit gracefully
- If planning fails → Exit gracefully
- If research fails → Continue with available data
- If synthesis fails → Show error message

This makes the system **robust** and **user-friendly**.


In [17]:
def run_research_assistant(topic: str, api_key: str = None) -> dict:
    """
    Main orchestrator function that coordinates all agents.
    
    This function demonstrates the complete multi-agent workflow:
    1. Configuration
    2. Planning
    3. Research (parallelizable in future)
    4. Synthesis
    5. Output
    
    Args:
        topic: The research topic
        api_key: Optional API key (if not using environment variable)
    
    Returns:
        Dictionary with:
        - 'success': bool
        - 'topic': str
        - 'plan': list of questions
        - 'research_results': list of (question, data) tuples
        - 'report': final report string
        - 'error': error message if failed
    """
    result = {
        'success': False,
        'topic': topic,
        'plan': [],
        'research_results': [],
        'report': '',
        'error': None
    }
    
    try:
        # Step 1: Configure the client
        print("🔧 Step 1: Configuring Gemini API...")
        client = configure_gemini(api_key=api_key)
        print("✅ Configuration successful!\n")
        
    except ValueError as e:
        error_msg = str(e)
        print(f"❌ Configuration Error: {error_msg}")
        result['error'] = error_msg
        return result
    except Exception as e:
        error_msg = f"Unexpected configuration error: {e}"
        print(f"❌ {error_msg}")
        result['error'] = error_msg
        return result
    
    # Step 2: Validate input
    if not topic or not topic.strip():
        error_msg = "Topic cannot be empty"
        print(f"❌ {error_msg}")
        result['error'] = error_msg
        return result
    
    print(f"📚 Starting research process for: '{topic}'\n")
    print("=" * 70)
    
    # Step 3: Planning phase
    print("\n🎯 Phase 1: Planning")
    print("-" * 70)
    research_plan = planner_agent(client, topic)
    result['plan'] = research_plan
    
    if not research_plan:
        error_msg = "Could not create a research plan"
        print(f"\n❌ {error_msg}")
        result['error'] = error_msg
        return result
    
    # Step 4: Research phase
    print(f"\n🔍 Phase 2: Research ({len(research_plan)} questions)")
    print("-" * 70)
    research_results = []
    
    for i, question in enumerate(research_plan, 1):
        print(f"\n[{i}/{len(research_plan)}] ", end="")
        research_data = search_agent(client, question)
        if research_data:
            research_results.append((question, research_data))
        else:
            print(f"   ⚠️  Skipping question due to error")
    
    result['research_results'] = research_results
    
    if not research_results:
        error_msg = "Could not find any information during research"
        print(f"\n❌ {error_msg}")
        result['error'] = error_msg
        return result
    
    # Step 5: Synthesis phase
    print(f"\n✍️  Phase 3: Synthesis")
    print("-" * 70)
    final_report = synthesizer_agent(client, topic, research_results)
    result['report'] = final_report
    
    if not final_report or "Error:" in final_report:
        error_msg = "Could not generate final report"
        print(f"\n❌ {error_msg}")
        result['error'] = error_msg
        return result
    
    # Success!
    result['success'] = True
    print("\n" + "=" * 70)
    print("✅ Research completed successfully!")
    print("=" * 70)
    
    return result


def display_report(result: dict):
    """
    Display the final research report in a formatted way.
    
    Args:
        result: Result dictionary from run_research_assistant()
    """
    if not result['success']:
        print(f"\n❌ Research failed: {result.get('error', 'Unknown error')}")
        return
    
    print("\n" + "=" * 70)
    print("📄 FINAL RESEARCH REPORT")
    print("=" * 70)
    print(f"\n## Topic: {result['topic']}\n")
    print(result['report'])
    print("\n" + "=" * 70)
    print("--- END OF REPORT ---")
    print("=" * 70)


In [18]:
# Example 1: Test with a technology topic
topic = "Large Language Models and their applications in healthcare"

print("🚀 Running Complete Multi-Agent Research System")
print("=" * 70)
print(f"Topic: {topic}\n")

result = run_research_assistant(topic)

# Display the results
display_report(result)


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


🚀 Running Complete Multi-Agent Research System
Topic: Large Language Models and their applications in healthcare

🔧 Step 1: Configuring Gemini API...
✅ API key configured from environment variable
✅ Configuration successful!

📚 Starting research process for: 'Large Language Models and their applications in healthcare'


🎯 Phase 1: Planning
----------------------------------------------------------------------
📋 Planner Agent: Creating a research plan...
   Using model: gemini-2.5-flash
✅ Plan created with 3 questions:
   1. What are the primary current applications of Large Language Models in clinical decision support and patient care?
   2. What are the key benefits and potential advantages of integrating Large Language Models into medical research and drug discovery processes?
   3. What are the significant ethical considerations and technical challenges related to the development and deployment of Large Language Models in healthcare environments?

🔍 Phase 2: Research (3 questions)
-

### 🎓 Try Your Own Topic

Modify the cell below to research any topic you're interested in:


In [19]:
# 🎯 YOUR TURN: Research your own topic!
# Change the topic below to anything you want to research

your_topic = "Attack on Venezuela and its impact on the economy"

print("🚀 Running Complete Multi-Agent Research System")
print("=" * 70)
print(f"Topic: {your_topic}\n")

result = run_research_assistant(your_topic)
display_report(result)


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


🚀 Running Complete Multi-Agent Research System
Topic: Attack on Venezuela and its impact on the economy

🔧 Step 1: Configuring Gemini API...
✅ API key configured from environment variable
✅ Configuration successful!

📚 Starting research process for: 'Attack on Venezuela and its impact on the economy'


🎯 Phase 1: Planning
----------------------------------------------------------------------
📋 Planner Agent: Creating a research plan...
   Using model: gemini-2.5-flash
✅ Plan created with 6 questions:
   1. What are the specific international economic sanctions imposed on Venezuela since 2017
   2. and which key economic sectors have been most affected?
   3. How have international economic sanctions impacted Venezuelas GDP
   4. inflation rates
   5. and and oil production between 2017 and 2023?
   6. What has been the effect of foreign-supported political destabilization efforts on foreign direct investment and capital flight in Venezuela since 2015?

🔍 Phase 2: Research (6 questions)

Traceback (most recent call last):
  File "/var/folders/kn/4r8ws4g95q1dwsv9btm_lc000000gn/T/ipykernel_47542/1775844127.py", line 56, in search_agent
    response = client.models.generate_content(
        model=model_name,
        contents=prompt,
        config=config
    )
  File "/Users/balaji/Documents/Learning/AI/ai_agent_projects/venv/lib/python3.13/site-packages/google/genai/models.py", line 5203, in generate_content
    response = self._generate_content(
        model=model, contents=contents, config=parsed_config
    )
  File "/Users/balaji/Documents/Learning/AI/ai_agent_projects/venv/lib/python3.13/site-packages/google/genai/models.py", line 3985, in _generate_content
    response = self._api_client.request(
        'post', path, request_dict, http_options
    )
  File "/Users/balaji/Documents/Learning/AI/ai_agent_projects/venv/lib/python3.13/site-packages/google/genai/_api_client.py", line 1388, in request
    response = self._request(http_request, http_options, stream=False)

   ❌ Error in Researcher Agent: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 52.888575242s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-f

Traceback (most recent call last):
  File "/var/folders/kn/4r8ws4g95q1dwsv9btm_lc000000gn/T/ipykernel_47542/1775844127.py", line 56, in search_agent
    response = client.models.generate_content(
        model=model_name,
        contents=prompt,
        config=config
    )
  File "/Users/balaji/Documents/Learning/AI/ai_agent_projects/venv/lib/python3.13/site-packages/google/genai/models.py", line 5203, in generate_content
    response = self._generate_content(
        model=model, contents=contents, config=parsed_config
    )
  File "/Users/balaji/Documents/Learning/AI/ai_agent_projects/venv/lib/python3.13/site-packages/google/genai/models.py", line 3985, in _generate_content
    response = self._api_client.request(
        'post', path, request_dict, http_options
    )
  File "/Users/balaji/Documents/Learning/AI/ai_agent_projects/venv/lib/python3.13/site-packages/google/genai/_api_client.py", line 1388, in request
    response = self._request(http_request, http_options, stream=False)

   ❌ Error in Researcher Agent: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 52.610391309s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 

Traceback (most recent call last):
  File "/var/folders/kn/4r8ws4g95q1dwsv9btm_lc000000gn/T/ipykernel_47542/3168451373.py", line 78, in synthesizer_agent
    response = client.models.generate_content(
        model=model_name,
        contents=prompt
    )
  File "/Users/balaji/Documents/Learning/AI/ai_agent_projects/venv/lib/python3.13/site-packages/google/genai/models.py", line 5203, in generate_content
    response = self._generate_content(
        model=model, contents=contents, config=parsed_config
    )
  File "/Users/balaji/Documents/Learning/AI/ai_agent_projects/venv/lib/python3.13/site-packages/google/genai/models.py", line 3985, in _generate_content
    response = self._api_client.request(
        'post', path, request_dict, http_options
    )
  File "/Users/balaji/Documents/Learning/AI/ai_agent_projects/venv/lib/python3.13/site-packages/google/genai/_api_client.py", line 1388, in request
    response = self._request(http_request, http_options, stream=False)
  File "/Users/ba

## 🧪 Test Cases and Scenarios

Testing is crucial for building reliable systems. Let's create comprehensive test cases to validate our multi-agent system works correctly in various scenarios.


In [ ]:
def test_planner_agent(client, test_cases):
    """Test the Planner Agent with various topics."""
    print("🧪 Testing Planner Agent\n")
    print("=" * 70)
    
    results = []
    for i, topic in enumerate(test_cases, 1):
        print(f"\nTest {i}: '{topic}'")
        print("-" * 70)
        questions = planner_agent(client, topic)
        results.append({
            'topic': topic,
            'questions': questions,
            'success': len(questions) >= 3
        })
        print(f"✅ Generated {len(questions)} questions" if questions else "❌ Failed")
    
    return results


def test_researcher_agent(client, test_questions):
    """Test the Researcher Agent with various questions."""
    print("\n🧪 Testing Researcher Agent\n")
    print("=" * 70)
    
    results = []
    for i, question in enumerate(test_questions, 1):
        print(f"\nTest {i}: '{question[:60]}...'")
        print("-" * 70)
        answer = search_agent(client, question)
        results.append({
            'question': question,
            'answer_length': len(answer),
            'success': len(answer) > 100  # At least 100 characters
        })
        print(f"✅ Retrieved {len(answer)} characters" if answer else "❌ Failed")
    
    return results


def test_synthesizer_agent(client, test_data):
    """Test the Synthesizer Agent with sample research data."""
    print("\n🧪 Testing Synthesizer Agent\n")
    print("=" * 70)
    
    topic = test_data['topic']
    research_results = test_data['research_results']
    
    print(f"Topic: {topic}")
    print(f"Research Results: {len(research_results)} items")
    print("-" * 70)
    
    report = synthesizer_agent(client, topic, research_results)
    
    result = {
        'topic': topic,
        'report_length': len(report),
        'success': len(report) > 200 and "Error:" not in report
    }
    
    print(f"✅ Generated {len(report)} character report" if result['success'] else "❌ Failed")
    
    return result


def run_all_tests():
    """Run comprehensive test suite."""
    print("🧪 COMPREHENSIVE TEST SUITE")
    print("=" * 70)
    print("This will test all agents with various scenarios.\n")
    
    try:
        client = configure_gemini()
    except Exception as e:
        print(f"❌ Cannot run tests: {e}")
        return
    
    # Test Case 1: Planner Agent - Various Topics
    planner_test_cases = [
        "Climate change impacts",
        "Artificial Intelligence ethics",
        "Renewable energy technologies"
    ]
    planner_results = test_planner_agent(client, planner_test_cases)
    
    # Test Case 2: Researcher Agent - Various Questions
    researcher_test_questions = [
        "What is machine learning?",
        "How does solar energy work?",
        "What are the benefits of electric vehicles?"
    ]
    researcher_results = test_researcher_agent(client, researcher_test_questions)
    
    # Test Case 3: Synthesizer Agent - Sample Data
    synthesizer_test_data = {
        'topic': 'Artificial Intelligence',
        'research_results': [
            ("What is AI?", "Artificial Intelligence is the simulation of human intelligence..."),
            ("How is AI used?", "AI is used in healthcare, finance, transportation..."),
            ("What are AI risks?", "AI risks include job displacement, bias, privacy concerns...")
        ]
    }
    synthesizer_result = test_synthesizer_agent(client, synthesizer_test_data)
    
    # Summary
    print("\n" + "=" * 70)
    print("📊 TEST SUMMARY")
    print("=" * 70)
    print(f"Planner Agent: {sum(r['success'] for r in planner_results)}/{len(planner_results)} passed")
    print(f"Researcher Agent: {sum(r['success'] for r in researcher_results)}/{len(researcher_results)} passed")
    print(f"Synthesizer Agent: {'✅ Passed' if synthesizer_result['success'] else '❌ Failed'}")
    print("=" * 70)

# Uncomment to run tests (uses API credits)
# run_all_tests()


## 📋 Additional Scenarios

Let's explore different scenarios to understand how the system behaves in various situations:


In [ ]:
# Scenario 1: Current Events (Requires up-to-date information)
print("📰 Scenario 1: Current Events Research")
print("=" * 70)
current_event_topic = "Latest developments in space exploration in 2024"
result1 = run_research_assistant(current_event_topic)
if result1['success']:
    print(f"\n✅ Successfully researched current events topic")
    print(f"   Generated {len(result1['plan'])} research questions")
    print(f"   Collected {len(result1['research_results'])} research results")
    print(f"   Final report: {len(result1['report'])} characters")
else:
    print(f"\n❌ Failed: {result1.get('error', 'Unknown error')}")


In [ ]:
# Scenario 2: Technical/Complex Topic
print("\n🔬 Scenario 2: Technical/Complex Topic")
print("=" * 70)
technical_topic = "Quantum computing algorithms and their applications in cryptography"
result2 = run_research_assistant(technical_topic)
if result2['success']:
    print(f"\n✅ Successfully researched technical topic")
    print(f"   Generated {len(result2['plan'])} research questions")
    print(f"   Collected {len(result2['research_results'])} research results")
    print(f"   Final report: {len(result2['report'])} characters")
else:
    print(f"\n❌ Failed: {result2.get('error', 'Unknown error')}")


In [ ]:
# Scenario 3: Business/Economic Topic
print("\n💼 Scenario 3: Business/Economic Topic")
print("=" * 70)
business_topic = "Impact of remote work on urban real estate markets"
result3 = run_research_assistant(business_topic)
if result3['success']:
    print(f"\n✅ Successfully researched business topic")
    print(f"   Generated {len(result3['plan'])} research questions")
    print(f"   Collected {len(result3['research_results'])} research results")
    print(f"   Final report: {len(result3['report'])} characters")
else:
    print(f"\n❌ Failed: {result3.get('error', 'Unknown error')}")


## 🏗️ Full-Stack Solution Architecture Plan

Now that you understand how multi-agent systems work, let's design a **production-ready, full-stack solution** for a real-world application.

### 🎯 Real-World Example: AI-Powered Research Platform

Imagine building a platform where users can:
- Submit research topics
- Get comprehensive research reports
- Save and manage their research history
- Share reports with teams
- Export reports in various formats (PDF, DOCX, Markdown)

### 📐 System Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                        Frontend Layer                        │
│  ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌──────────┐  │
│  │   Web    │  │  Mobile  │  │  Desktop │  │   API    │  │
│  │   App    │  │   App    │  │   App    │  │  Client   │  │
│  └────┬─────┘  └────┬─────┘  └────┬─────┘  └────┬─────┘  │
└───────┼──────────────┼──────────────┼──────────────┼────────┘
        │              │              │              │
        └──────────────┴──────────────┴──────────────┘
                       │
        ┌──────────────▼──────────────┐
        │      API Gateway            │
        │  (Authentication, Rate      │
        │   Limiting, Load Balancing) │
        └──────────────┬──────────────┘
                       │
        ┌──────────────▼──────────────┐
        │    Backend Services         │
        │  ┌────────────────────────┐ │
        │  │  Research Service      │ │ ← Our Multi-Agent System
        │  │  - Planner Agent       │ │
        │  │  - Researcher Agent    │ │
        │  │  - Synthesizer Agent   │ │
        │  └────────────────────────┘ │
        │  ┌────────────────────────┐ │
        │  │  User Service         │ │
        │  │  - Authentication     │ │
        │  │  - Profile Management│ │
        │  └────────────────────────┘ │
        │  ┌────────────────────────┐ │
        │  │  Report Service       │ │
        │  │  - Storage            │ │
        │  │  - Export             │ │
        │  │  - Sharing            │ │
        │  └────────────────────────┘ │
        └──────────────┬──────────────┘
                       │
        ┌──────────────▼──────────────┐
        │    Data Layer               │
        │  ┌──────────┐  ┌──────────┐│
        │  │PostgreSQL│  │  Redis   ││
        │  │(Users,   │  │(Cache,   ││
        │  │ Reports) │  │  Queue)  ││
        │  └──────────┘  └──────────┘│
        │  ┌──────────┐  ┌──────────┐│
        │  │  S3/     │  │  Vector  ││
        │  │  GCS    │  │   DB     ││
        │  │(Files)  │  │(Search)  ││
        │  └──────────┘  └──────────┘│
        └─────────────────────────────┘
                       │
        ┌──────────────▼──────────────┐
        │    External Services        │
        │  ┌──────────┐  ┌──────────┐│
        │  │  Gemini  │  │  Google  ││
        │  │   API    │  │  Search  ││
        │  └──────────┘  └──────────┘│
        └─────────────────────────────┘
```

### 🛠️ Technology Stack Recommendations

#### Frontend
- **Web**: React/Next.js or Vue.js
- **Mobile**: React Native or Flutter
- **Desktop**: Electron (for cross-platform)

#### Backend
- **API Framework**: FastAPI (Python) or Express.js (Node.js)
- **Task Queue**: Celery (Python) or Bull (Node.js) for async processing
- **WebSockets**: For real-time progress updates

#### Database
- **Primary DB**: PostgreSQL (users, reports, metadata)
- **Cache**: Redis (session, rate limiting, queue)
- **Vector DB**: Pinecone or Weaviate (for semantic search of reports)

#### Infrastructure
- **Cloud**: AWS, GCP, or Azure
- **Containerization**: Docker + Kubernetes
- **CI/CD**: GitHub Actions or GitLab CI
- **Monitoring**: Prometheus + Grafana

### 📝 Implementation Phases

#### Phase 1: Core Backend (Weeks 1-2)
1. Set up FastAPI project structure
2. Implement authentication (JWT tokens)
3. Port multi-agent system to async service
4. Create database models (User, Report, ResearchTask)
5. Implement basic API endpoints

#### Phase 2: Research Service Enhancement (Weeks 3-4)
1. Add task queue for async research processing
2. Implement progress tracking (WebSockets)
3. Add research result caching
4. Implement rate limiting per user
5. Add error handling and retry logic

#### Phase 3: Frontend Development (Weeks 5-6)
1. Create React/Next.js frontend
2. Implement authentication UI
3. Build research submission form
4. Create real-time progress dashboard
5. Design report display and export features

#### Phase 4: Advanced Features (Weeks 7-8)
1. Add report templates and customization
2. Implement report sharing and collaboration
3. Add user dashboard with research history
4. Implement search across saved reports
5. Add export to PDF/DOCX/Markdown

#### Phase 5: Production Readiness (Weeks 9-10)
1. Add comprehensive logging and monitoring
2. Implement automated testing (unit, integration, E2E)
3. Set up CI/CD pipeline
4. Performance optimization and load testing
5. Security audit and hardening
6. Documentation and deployment guides

### 🔐 Security Considerations

1. **API Key Management**
   - Store Gemini API keys in secure vault (AWS Secrets Manager, HashiCorp Vault)
   - Rotate keys regularly
   - Use different keys for dev/staging/prod

2. **Authentication & Authorization**
   - JWT tokens with refresh tokens
   - Role-based access control (RBAC)
   - Rate limiting per user tier

3. **Data Protection**
   - Encrypt sensitive data at rest
   - Use HTTPS for all communications
   - Implement input validation and sanitization
   - Protect against SQL injection, XSS attacks

4. **API Security**
   - API rate limiting
   - Request validation
   - CORS configuration
   - API versioning

### 📊 Scalability Considerations

1. **Horizontal Scaling**
   - Stateless API services (can scale horizontally)
   - Load balancer for distribution
   - Database connection pooling

2. **Caching Strategy**
   - Cache research results for common topics
   - Cache user sessions
   - Cache frequently accessed reports

3. **Async Processing**
   - Use task queues for long-running research tasks
   - Implement job prioritization
   - Add retry mechanisms with exponential backoff

4. **Database Optimization**
   - Index frequently queried fields
   - Implement database replication for read scaling
   - Use connection pooling

### 💰 Cost Optimization

1. **API Usage**
   - Cache research results to avoid duplicate API calls
   - Implement smart rate limiting
   - Use cheaper models for simple tasks

2. **Infrastructure**
   - Use auto-scaling to match demand
   - Implement cost monitoring and alerts
   - Use reserved instances for predictable workloads

3. **Storage**
   - Implement data retention policies
   - Compress old reports
   - Use tiered storage (hot/cold)

### 🧪 Testing Strategy

1. **Unit Tests**: Test each agent independently
2. **Integration Tests**: Test agent orchestration
3. **E2E Tests**: Test complete user workflows
4. **Load Tests**: Test system under high load
5. **Security Tests**: Penetration testing, vulnerability scanning

### 📈 Monitoring & Observability

1. **Metrics**
   - API response times
   - Research task completion rates
   - Error rates by agent
   - API usage and costs

2. **Logging**
   - Structured logging (JSON format)
   - Log aggregation (ELK stack or similar)
   - Error tracking (Sentry)

3. **Alerting**
   - API errors above threshold
   - High latency alerts
   - Cost overruns
   - System downtime

### 🚀 Deployment Strategy

1. **Environments**
   - Development (local)
   - Staging (mirrors production)
   - Production

2. **Deployment Method**
   - Blue-green deployment for zero downtime
   - Canary releases for gradual rollout
   - Feature flags for A/B testing

3. **Disaster Recovery**
   - Regular database backups
   - Multi-region deployment
   - Automated failover

---

## 🎓 Next Steps for Learning

1. **Extend the Current System**
   - Add more agent types (Fact Checker, Citation Manager)
   - Implement parallel research (multiple questions simultaneously)
   - Add agent memory/context sharing

2. **Learn Advanced Concepts**
   - LangGraph for complex agent workflows
   - Agent memory and state management
   - Tool calling and function execution
   - Multi-modal agents (text + images)

3. **Build Your Own Project**
   - Choose a domain (legal research, medical literature, etc.)
   - Design your agent architecture
   - Implement and iterate
   - Deploy to production

4. **Join the Community**
   - Contribute to open-source multi-agent projects
   - Share your learnings
   - Participate in AI agent competitions

---

## 📚 Additional Resources

- [Google Gemini API Documentation](https://ai.google.dev/docs)
- [Multi-Agent Systems Research](https://arxiv.org/search/?query=multi-agent+systems)
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [FastAPI Documentation](https://fastapi.tiangolo.com/)
- [System Design Primer](https://github.com/donnemartin/system-design-primer)

---

## ✅ Summary

In this notebook, you've learned:

1. ✅ **API Key Integration**: Secure configuration for Colab and local environments
2. ✅ **Multi-Agent Architecture**: How to design specialized agents
3. ✅ **Agent Orchestration**: Coordinating multiple agents in a workflow
4. ✅ **Prompt Engineering**: Crafting effective prompts for specific roles
5. ✅ **Tool Integration**: Using external tools (Google Search) with LLMs
6. ✅ **Error Handling**: Building robust, production-ready systems
7. ✅ **Testing**: Creating comprehensive test cases
8. ✅ **Full-Stack Design**: Planning a complete production system

**Congratulations!** You now have the knowledge to build sophisticated multi-agent systems. The next step is to build your own project and experiment with different architectures!

---

*Happy Learning! 🚀*
